## Setup

In [1]:
!pip install -q holidays

In [2]:
import pandas as pd
import numpy as np

In [4]:
def load_stock_price_dataset(symbol):
    return pd.read_csv(
            f"../datasets/{symbol.lower()}.us.txt", 
            index_col='datetime', 
            parse_dates={'datetime': ['<DATE>', '<TIME>']},
            usecols=['<DATE>', '<TIME>', '<CLOSE>'], 
            na_values=['nan']
    ).rename(columns={'<CLOSE>': 'price'})   

In [5]:
import datetime

import holidays
import pandas as pd


def is_us_holiday(dt):
    return dt.strftime("%Y-%m-%d") in holidays.UnitedStates()


def extract_datetime_features(ds):
    df = pd.DataFrame()
    df.index = ds
    df["day"] = ds.day
    df["hour"] = ds.hour
    df["month_name"] = ds.month_name()
    df["day_name"] = ds.day_name()
    df["is_weekend"] = (ds.day_name() == 'Saturday') | (ds.day_of_week == 'Sunday')
    df["is_month_start"] = ds.is_month_start
    df["is_month_end"] = ds.is_month_end
    df["is_quarter_start"] = ds.is_quarter_start
    df["is_year_start"] = ds.is_year_start
    # US holidays
    df["is_holiday"] = pd.Series(ds.values).apply(is_us_holiday).values
    df["is_day_before_holiday"] = [is_us_holiday(dt) for dt in pd.Series(ds + datetime.timedelta(days=1)).tolist()]
    df["is_day_after_holiday"] = [is_us_holiday(dt) for dt in pd.Series(ds - datetime.timedelta(days=1)).tolist()]
    nominals = [
        "hour",
        "day",
        "day_name",
        "month_name",
        "is_weekend",
        "is_month_start",
        "is_month_end",
        "is_quarter_start",
        "is_year_start",
        "is_holiday",
        "is_day_before_holiday",
        "is_day_after_holiday",
    ]
    for col in nominals:
        if col in df.columns:
            df[col] = df[col].astype("category")

    return df


def add_datetime_features(df):
    return pd.concat([extract_datetime_features(df.index), df], axis=1)


def add_price_change(df):
    df["price_change"] = df.price.diff().fillna(0)
    return df


def order_cols(df):
    categorical_cols = df.select_dtypes("category").columns.tolist()
    numerical_cols = df.select_dtypes("float").columns.tolist()
    existing_cols = set(df.columns)
    col_order = [
        col for col in numerical_cols + categorical_cols if col in existing_cols
    ]
    return df[col_order]


def prepare_dataset(df):
    return df.pipe(add_datetime_features).pipe(add_price_change).pipe(order_cols)

In [7]:
SYMBOL = 'aapl'
df = (
    load_stock_price_dataset(SYMBOL)
    .pipe(prepare_dataset)
)
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1454 entries, 2022-03-04 16:00:00 to 2022-12-29 22:00:00
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   price                  1454 non-null   float64 
 1   price_change           1454 non-null   float64 
 2   day                    1454 non-null   category
 3   hour                   1454 non-null   category
 4   month_name             1454 non-null   category
 5   day_name               1454 non-null   category
 6   is_weekend             1454 non-null   category
 7   is_month_start         1454 non-null   category
 8   is_month_end           1454 non-null   category
 9   is_quarter_start       1454 non-null   category
 10  is_year_start          1454 non-null   category
 11  is_holiday             1454 non-null   category
 12  is_day_before_holiday  1454 non-null   category
 13  is_day_after_holiday   1454 non-null   category
dtypes: c

In [8]:
df.head()

,price,price_change,day,hour,month_name,day_name,is_weekend,is_month_start,is_month_end,is_quarter_start,is_year_start,is_holiday,is_day_before_holiday,is_day_after_holiday
datetime,,,,,,,,,,,,,,
2022-03-04 16:00:00,164.328,0.000,4,16,March,Friday,False,False,False,False,False,False,False,False
2022-03-04 17:00:00,161.847,-2.481,4,17,March,Friday,False,False,False,False,False,False,False,False
2022-03-04 18:00:00,162.016,0.169,4,18,March,Friday,False,False,False,False,False,False,False,False
2022-03-04 19:00:00,162.953,0.937,4,19,March,Friday,False,False,False,False,False,False,False,False
2022-03-04 20:00:00,162.385,-0.568,4,20,March,Friday,False,False,False,False,False,False,False,False


In [9]:
df.to_csv(f'../datasets/{SYMBOL}.us-datetime-features.csv')